In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  

In [ ]:
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
from agents.fairy.util import get_groq_llm_lc
from langchain_core.messages import HumanMessage, SystemMessage
from agents.fairy.fairy_state import FairyDungeonIntentOutput
from enums.LLM import LLM

expect_multi_question = "HP 포션좀 먹고 싶어, 물약좀 먹으면서 저 슬라임 공략법에 대해 알려줘"
expect_monster = "저거 공략법좀 알려줘"
expect_event = "이 석상을 부시면 어떻게 되는거야?"
expect_navigator= "현재 방에서 어디로 가야돼?"
expect_interaction_handler = "방 불좀 켜줘"
expect_small_talk = "심심행"
expect_unknown_intent = "al;sdas 내가 낀 아이템을 사용해"
expect_game_system = "게임 어떻게 꺼?"

expect_super_multi = "내가 낀 아이템을 사용해 그리고 저 몬스터 공략법좀 알려줘 그리고 이 석상을 어떻게 부셔? 다음 던전 방에는 뭐고? 그리고 심심해 그리고 게임 어떻게 종료해?"

system_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_INTENT).get_prompt()
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="뭙야")
] 

parser_llm = get_groq_llm_lc(model=LLM.LLAMA_3_1_8B_INSTANT, max_token=43).with_structured_output(FairyDungeonIntentOutput)
result = parser_llm.invoke(messages)
print(result)

In [ ]:
%%writefile fairy_dungeon_agent.py
from agents.fairy.fairy_state import (
    FairyDungeonIntentOutput,
    FairyDungeonState,
    FairyDungeonIntentType,
)
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.types import interrupt
from agents.fairy.cache_data import reverse_questions, GAME_SYSTEM_INFO
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
import random, asyncio
from agents.fairy.util import (
    add_ai_message,
    add_human_message,
    str_to_bool,
    get_groq_llm_lc,
    find_monsters_info,
)
from core.common import get_inventory_items
from enums.LLM import LLM
from langchain.chat_models import init_chat_model
from typing import List
from db.RDBRepository import RDBRepository
from db.rdb_entity.DungeonRow import DungeonRow
from agents.fairy.dynamic_prompt import dungeon_spec_prompt

intent_llm = get_groq_llm_lc(model=LLM.LLAMA_3_1_8B_INSTANT, max_token=43)
action_llm = get_groq_llm_lc(max_token=80,temperature=0)
small_talk_llm = init_chat_model(model=LLM.GROK_4_FAST_NON_REASONING)
rdb_repository = RDBRepository()

async def get_monsters_info(target_monster_ids: List[int]):
    return find_monsters_info(target_monster_ids)


async def get_event_info(dungeon_row: DungeonRow, curr_room_id: int):
    curr_room_id
    return dungeon_row.event


async def dungeon_navigator(dungeon_row: DungeonRow, curr_room_id: int):
    summary_info = dungeon_row.summary_info
    dungeon_json_prompt = dungeon_spec_prompt.format(
        balanced_map_json=dungeon_row.balanced_map
    )
    dungeon_map_prompt = f"        <던전맵>\n{dungeon_json_prompt}\n        </던전맵>"
    dungeon_summary_prompt = f"        <던전요약>\n{summary_info}\n        </던전요약>"
    dungeon_current_prompt = (
        f"        <현재 Room Id>\n{curr_room_id}\n        </현재 Room Id>"
    )
    total_prompt = (
        dungeon_map_prompt
        + "\n"
        + dungeon_summary_prompt
        + "\n"
        + dungeon_current_prompt
    )
    return total_prompt


async def create_interaction(inventory_ids):

    inventory_prompt = f"        <인벤토리 내의 아이템 설명>\n{get_inventory_items(inventory_ids)}\n        </인벤토리 내의 아이템 설명>"
    result = inventory_prompt
    return result


async def get_system_info():
    return GAME_SYSTEM_INFO


async def _clarify_intent(query) -> FairyDungeonIntentOutput:
    intent_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_INTENT).get_prompt()
    messages = [SystemMessage(content=intent_prompt), HumanMessage(content=query)]
    parser_llm = intent_llm.with_structured_output(FairyDungeonIntentOutput)
    intent_output: FairyDungeonIntentOutput = await parser_llm.ainvoke(messages)
    print("전체 의도::", intent_output)
    return intent_output


async def analyze_intent(state: FairyDungeonState):
    last = state["messages"][-1]
    last_message = last.content
    clarify_intent_type: FairyDungeonIntentOutput = await _clarify_intent(last_message)

    if clarify_intent_type.intents[0] == FairyDungeonIntentType.UNKNOWN_INTENT:
        clarification = reverse_questions[random.randint(0, 148)]
        user_resp = interrupt(clarification)
        return {
            "messages": [
                add_ai_message(
                    content=clarification, intent_types=clarify_intent_type.intents
                ),
                add_human_message(content=user_resp),
            ],
            "intent_types": clarify_intent_type.intents,
            "is_multi_small_talk": False,
        }

    return {
        "intent_types": clarify_intent_type.intents,
    }


def check_condition(state: FairyDungeonState):
    intent_types = state.get("intent_types", [])
    if intent_types[0] == FairyDungeonIntentType.UNKNOWN_INTENT:
        return "retry"
    return "continue"


async def fairy_action(state: FairyDungeonState):
    intent_types = state.get("intent_types")
    dungenon_player = state["dungenon_player"]
    target_monster_ids = state.get("target_monster_ids", [])
    currRoomId = dungenon_player.currRoomId
    dungeon_row = rdb_repository.get_current_dungeon_by_player(
        dungenon_player.playerId, dungenon_player.heroineId
    )
    messages = state["messages"]
    INTENT_HANDLERS = {
        FairyDungeonIntentType.MONSTER_GUIDE: lambda: get_monsters_info(
            target_monster_ids
        ),
        FairyDungeonIntentType.EVENT_GUIDE: lambda: get_event_info(
            dungeon_row, currRoomId
        ),
        FairyDungeonIntentType.DUNGEON_NAVIGATOR: lambda: dungeon_navigator(
            dungeon_row, currRoomId
        ),
        FairyDungeonIntentType.INTERACTION_HANDLER: lambda: create_interaction(
            dungenon_player.inventory
        ),
        FairyDungeonIntentType.USAGE_GUIDE: get_system_info,
    }

    INTENT_LABELS = {
        FairyDungeonIntentType.MONSTER_GUIDE: "몬스터 정보",
        FairyDungeonIntentType.EVENT_GUIDE: "이벤트",
        FairyDungeonIntentType.DUNGEON_NAVIGATOR: "던전 안내",
        FairyDungeonIntentType.INTERACTION_HANDLER: "상호작용",
        FairyDungeonIntentType.USAGE_GUIDE: "사용 방법·조작 안내",
    }

    handlers = [INTENT_HANDLERS[i]() for i in intent_types if i in INTENT_HANDLERS]
    results = await asyncio.gather(*handlers)

    prompt_info = ""
    idx = 0
    for i, index in enumerate(intent_types):
        handler = INTENT_HANDLERS.get(index)
        if not handler:
            continue

        value = results[idx]
        label = INTENT_LABELS.get(index, "정보")
        if i == 0:
            prompt_info += f"    <{label}>\n{value}\n    </{label}>"
        else:
            prompt_info += f"\n    <{label}>\n{value}\n    </{label}>"
        idx += 1

    pretty_dungenon_player = dungenon_player.model_dump_json(indent=2)
    system_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_SYSTEM).get_prompt()

    question = messages[-1].content
    human_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_HUMAN).get_prompt(
        dungenon_player=pretty_dungenon_player,
        use_intents=[rt.value if hasattr(rt, "value") else rt for rt in intent_types],
        info=prompt_info,
        question=question,
    )

    print("check_prompt::", human_prompt)

    if intent_types[0] == FairyDungeonIntentType.SMALLTALK and len(intent_types) == 1:
        ai_answer = small_talk_llm.invoke(
            [SystemMessage(content=system_prompt)]
            + messages
            + [HumanMessage(content=human_prompt)]
        )
    else:
        ai_answer = action_llm.invoke(
            [
                SystemMessage(content=system_prompt),
                HumanMessage(content=human_prompt),
            ]
        )
    return {
        "messages": [
            add_ai_message(content=ai_answer.content, intent_types=intent_types)
        ]
    }


from langgraph.graph import START, END, StateGraph

graph_builder = StateGraph(FairyDungeonState)

graph_builder.add_node("analyze_intent", analyze_intent)
graph_builder.add_node("fairy_action", fairy_action)
# graph_builder.add_node("multi_small_talk", multi_small_talk_node)

graph_builder.add_edge(START, "analyze_intent")

graph_builder.add_conditional_edges(
    "analyze_intent",
    check_condition,
    {
        "retry": "analyze_intent",
        "continue": "fairy_action",
    },
)
graph_builder.add_edge("fairy_action", END)

# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState
import random

temp_player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=1,
    difficulty=random.randint(0, 2),
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="던전을 돌면 뭐가 좋다 했지?")],    
    "dungenon_player": temp_player,
    "target_monster_ids":[0,1],
}
await graph.ainvoke(inputs, config=config)
await graph.ainvoke({
    "messages":[add_human_message("스킬 사용법이 뭐야?")]
}, config=config)
result = await graph.ainvoke({
    "messages":[add_human_message("내 아이템중에 가장 쎈 무기가 뭐야?")]
}, config=config)
result["messages"]

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [ ]:
await graph.ainvoke(
    { "messages": HumanMessage(content="저기 있는 몬스터는 뭐야?") },
    config=config
)
result = await graph.ainvoke(
    { "messages": HumanMessage(content="내가 잡을 수 있지?") },
    config=config
)
result["messages"]

## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [ ]:

temp_player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=2,
    difficulty=random.randint(0, 2),
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)
result = await graph.ainvoke(
    { "messages": HumanMessage(content="이벤트 어떻게 깨?") },
    config=config
)
result["messages"]

## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [ ]:
result = await graph.ainvoke(
    { "messages": HumanMessage(content="다음 방은 뭐가 나올까?") },
    config=config
)
result["messages"]

## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [ ]:
await graph.ainvoke(
    { "messages": HumanMessage(content="저기 있는 몬스터는 뭐야?") },
    config=config
)
result = await graph.ainvoke(
    { "messages": HumanMessage(content="내가 잡을 수 있지?") },
    config=config
)
result["messages"]

In [ ]:
await graph.ainvoke(
    { "messages": HumanMessage(content="내가 맨 처음에 질문했던 내용은 뭐였지? ㅎㅎ") },
    config=config
)

In [ ]:
await graph.ainvoke(
    { "messages": HumanMessage(content="던전을 왜 돌아야돼?") },
    config=config
)

In [ ]:
await graph.ainvoke(
    { "messages": HumanMessage(content="다음 방에 대한 정보좀 줘") },
    config=config
)

In [ ]:
await graph.ainvoke(
    { "messages": HumanMessage(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?") },
    config=config
)